In [ ]:
import pandas as pd
import sys

In [ ]:
tbl = pd.read_csv("test.smbtbl", delimiter='|')
tbl.set_index("MLo", inplace=True)
print(tbl.columns)
tbl

In [ ]:
file_path = 'test.quad'
commands = []
with open(file_path, 'r') as file:
    for line in file:
        quad = line.strip('(')
        quads = quad[:-2].split(', ')
        commands.append(quads)
print(commands[0][3])

In [ ]:
# for command in commands:
#     print(command)

In [ ]:
for i, command in enumerate(commands):
    if command[0] == "CALL":
        aux = command[1]
        command[1] = "$t28"
        print(command)
        for comando in commands[i:]:
            for j in range(1,4):
                if comando[j] == aux:
                    comando[j] = "$t28"
                    print(comando)

usedIndexes = []
def fixIndex(line):
    global usedIndexes
    tobeUsed = []
    for command in commands[line + 1:]:
        # index = command[1].strip("$t")
        for i in range(1,len(command)):
            if "$t" in command[i]:
                #print(command[i])
                if command[i][2:] in usedIndexes and command[i][2:] not in tobeUsed:
                    tobeUsed.append(command[i][2:])
    print(usedIndexes)
    print("\n")
    usedIndexes = list(set(usedIndexes) - set(set(usedIndexes) - set(tobeUsed)))

# tobeUsed

In [ ]:
# for command in commands:
#     print(command)

In [ ]:
tbl.index[tbl["Name"] == "main"][0]

In [ ]:
def getIndex(reg, indice, indice2):
    if "$t" in commands[indice][indice2]:
        index = int(reg[2:])
        print("index = ",index)
        if index == 0:
            print("old = ",reg, "|returned = ",commands[indice][indice2])
            return commands[indice][indice2]
        if index < 28:
            print("old = ",reg, "|returned = ",commands[indice][indice2])
            return commands[indice][indice2]
        for num in range(1, 28):
            print(str(num),usedIndexes)
            if str(num) not in usedIndexes:
                #print(i, " Esta disponivel\n")
                for i in range(0,len(commands)):
                    for j in range(1,len(commands[i])):
                        if "$t" in commands[i][j]:
                            print("for ",commands[i][j], reg)
                            if commands[i][j] == reg:
                                commands[i][j] = "$t"+str(num)
                                print("Novo reg = ","$t"+str(num))
                                break
                print("old = ",reg, "|returned = ","$t"+str(num))
                print("REMOVED = ",index, "|appended = ","$t"+str(num))
                print(usedIndexes)
                usedIndexes.remove(str(index))
                usedIndexes.append(str(num))
                return "$t"+str(num)
        print("\n----------------------------\nFATAL ERROR: NOT ENOUGH REGISTERS!!!\n----------------------------\n")
    return "FATAL ERROR"

In [ ]:
tbl[(tbl['Name'] == 'v') & (tbl['Scope'] == 'gcd')].index[0]

In [ ]:
#addi beq bne bgt blt lw sw j jal (jar) add sub div mult and or
usedVars = {}
Params = []
#usedRegs = []
assembly = open("test.asm", 'w')
scope = " "
Callflag = 0
assembly.write("j "+"main""\n")
for i, command in enumerate(commands):
    for reg in command[1:]:
        if "$t" in reg and reg[2:] not in usedIndexes:
            usedIndexes.append(reg[2:])
    print(command)
    match command[0]:
        case 'FUNC':
            #assembly.write(command[2]+"\n")
            scope = command[2]
            assembly.write("addi $t30 $t30 1\n")
            assembly.write("sw $t31 $t29 1"+"\n")
        case 'END':
            assembly.write("lw $t31 $t29 1"+"\n")
            assembly.write("jr $t31"+"\n")
        case 'ARG':
            assembly.write("addi $t30 $t30 1\n")
        case 'LOAD':
            reg = getIndex(command[1],i,1)
            if command[2] not in usedVars:
                usedVars[command[2]] = reg
            assembly.write("lw "+reg+" "+"$t29 "+str(tbl[(tbl['Name'] == command[2]) & (tbl['Scope'] == scope)].index[0])+"\n")
        case 'GOTO':
            assembly.write("j "+command[1]+"\n")
        case 'LAB':
            assembly.write("LAB "+command[1]+"\n")
        case 'WHILE':
            assembly.write("LAB "+command[1]+"\n")
        case 'RET':
            if command[1] == "$t28":
                assembly.write("addi $t28 "+ "$t28 "+"0"+"\n")
            else:
                assembly.write("addi $t28 "+ getIndex(command[1],i,1)+" 0"+"\n")
        case 'END':
            if(command[1] != "main"):
                assembly.write("lw $t31 $t29 1\n")
        case 'CALL':
            #assembly.write(command[1]+"\n")
            #assembly.write("CALL BEGINS"+"\n")
            if command[2] == "input":
                assembly.write("input " +"$t28"+"\n")
            elif command[2] == "output":
                assembly.write("output " +"$t28"+"\n")
            else:
                backup = usedVars.copy()
                for var in list(usedVars):
                    assembly.write("sw "+usedVars[var]+" "+"$t30 "+ "0"+"\n")
                    assembly.write("addi " +"$t30 " +"$t30 "+ "1"+"\n")
                    del usedVars[var]
                assembly.write("sw " +"$t29 " +"$t30 "+ "0"+"\n")
                assembly.write("addi " +"$t29 " +"$t30 "+ "0"+"\n")
                assembly.write("addi " +"$t30 " +"$t30 "+ "1"+"\n")
                Mem = 2
                for param in Params:
                    assembly.write("sw "+param+" $t29 "+str(Mem)+"\n")
                    Mem += 1
                Mem = 2
                Params = []
                assembly.write("jal " +command[2]+"\n")
                assembly.write("addi " +"$t30 " +"$t29 "+ "0"+"\n")
                assembly.write("lw " +"$t29 " +"$t29 "+ "0"+"\n")
                usedVars = backup
                for var in list(usedVars):
                    assembly.write("subi " +"$t30 " +"$t30 "+ "1"+"\n")
                    assembly.write("lw "+usedVars[var]+" "+"$t30 "+ "0"+"\n")
                    del usedVars[var]
            #assembly.write("CALL ENDS"+"\n")
        case 'ADDI':
            #command[2] is always $t0
            assembly.write("addi "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+command[3]+"\n")
        case 'ALLOC':
            assembly.write("addi $t30 $t30 1\n")
        case 'STORE':
            assembly.write("sw "+getIndex(command[2],i,2) +" $t29 "+str(tbl[(tbl['Name'] == command[1]) & (tbl['Scope'] == scope)].index[0])+"\n")
        case 'PARAM':
            Params.append(command[1])
        case 'SOM':
            assembly.write("add "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'SUB':
            assembly.write("sub "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'MULT':
            assembly.write("mult "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'DIV':
            assembly.write("div "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'LT':
            assembly.write("lt "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'LET':
            assembly.write("let "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'GT':
            assembly.write("gt "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'GET':
            assembly.write("get "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'COMP':
            assembly.write("comp "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'NEQ':
            assembly.write("neq "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
    fixIndex(i)
#print(usedIndexes)
assembly.close()